# App Log Cleaning

In [1]:
from pathlib import Path
import json

import pandas as pd
import numpy as np
import pingouin as pg

import biopsykit as bp
from biopsykit.utils.time import time_to_timedelta
from biopsykit.utils.dataframe_handling import multi_xs, int_from_str_idx, camel_to_snake

from fau_colors import cmaps

import matplotlib.pyplot as plt
import seaborn as sns

from carwatch_analysis.datasets import CarWatchDatasetProcessed

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

pg.options["round"] = 4

palette

[(0.0, 0.1843137254901961, 0.4235294117647059),
 (0.4666666666666667, 0.6235294117647059, 0.7098039215686275),
 (1.0, 0.7215686274509804, 0.10980392156862745),
 (0.0, 0.6392156862745098, 0.8784313725490196),
 (0.2627450980392157, 0.6901960784313725, 0.16470588235294117),
 (0.7843137254901961, 0.06274509803921569, 0.1803921568627451)]

## Setup Paths

In [3]:
deploy_type = "develop"

In [4]:
# build path to data folder
config_dict = json.load(Path("../../../config.json").open(encoding="utf-8"))
base_path = Path("..").joinpath(config_dict[deploy_type]["base_path"])
base_path

PosixPath('/Users/Richer/Documents/PhD/Projects/HealthPsychology/CARWatch/Data')

In [5]:
dataset = CarWatchDatasetProcessed(base_path)
dataset

,subject,night
0,AB19E,0
1,AB19E,1
2,AB31R,0
3,AB31R,1
4,AC12E,0
...,...,...
229,VE19A,1
230,VS09S,0
231,VS09S,1
232,WM13K,0


## Load Data

### App Logs

In [8]:
dataset.get_subset(subject="AB19E").app_logs

,time,action,extras
0,2019-11-18 14:00:45.577000+01:00,subject_id_set,"{""subject_id"": ""AB19E"", ""subject_condition"": ""..."
1,2019-11-18 14:01:20.425000+01:00,subject_id_set,"{""subject_id"": ""AB19E"", ""subject_condition"": ""..."
2,2019-11-18 22:38:32.227000+01:00,evening_salivette,"{""id"": 815}"
3,2019-11-18 22:38:48.230000+01:00,barcode_scanned,"{""id"": 815, ""saliva_id"": 5, ""barcode_value"": ""..."
4,2019-11-18 22:38:54.706000+01:00,service_started,{}
...,...,...,...
83,2019-11-21 12:46:28.314000+01:00,screen_on,{}
84,2019-11-21 12:46:30.024000+01:00,user_present,{}
85,2019-11-21 13:16:22.114000+01:00,screen_off,{}
86,2019-11-21 13:18:20.275000+01:00,screen_on,{}


### Saliva

In [ ]:
cort_samples = dataset.cortisol_samples
cort_samples.head()

In [ ]:
cort_features = dataset.cortisol_features
cort_features.head()

## Data Cleaning

In [ ]:
print(f"Number of CARs before cleaning: {cort_samples.unstack('sample').shape[0]}")

### Remove CARs with any missing cortisol values

In [ ]:
cort_samples = clean_missing_values(cort_samples)

### Remove CARs with missing date information
Missing date information is defined as no valid data for any of:
* recording date
* wake onset
* sample time information

In [ ]:
cort_samples = clean_missing_date_information(cort_samples)

### Remove CARs with Differences >5 min between Wake Onset and S0

In [ ]:
cort_samples = clean_s0_after_wake_onset(cort_samples)

### Remove CARs with absolute difference between two consecutive saliva samples of >5 min from the actual time

In [ ]:
cort_samples = clean_sampling_time_difference(cort_samples)

### Remove Statistical Outlier ($> 3 \sigma$)

Remove CARs where any cortisol sample differs more than 3 standard deviations from the mean

In [ ]:
cort_samples = clean_statistical_outlier(cort_samples)

### Remove Physiological Outlier (Cortisol > 70 nmol/l)

In [ ]:
cort_samples = clean_physiological_outlier(cort_samples)

In [ ]:
cort_samples.head()

## Apply Index to Cortisol Features

In [ ]:
cort_features = cort_features.unstack("saliva_feature").reindex(cort_samples.unstack("sample").index)
cort_features = cort_features.stack()
cort_features.head()

## Adding Categorial Variables

### Wakeup Sources

In [ ]:
wakeup_source = dataset.questionnaire["wakeup_source"].fillna(0).astype(int)

### Weekend

In [ ]:
# NOTE: 4 = Friday, 5 = Saturday; this is chosen because "date" corresponds to the day when the night *started*,
# i.e, the nights from Friday to Saturday and from Saturday to Sunday are considered weekend
weekend = dataset.date["date"].dt.weekday.isin([4, 5]).astype(int)
weekend.name = "weekend"

### Wakeup Hour

In [ ]:
wakeup_hour = np.floor(dataset.sleep_information_merged["wake_onset_time"] / pd.Timedelta(hours=1))
wakeup_hour.name = "wakeup_hour"

### Chronotype

In [ ]:
meq = sleep_info["MEQ"]
chronotype = sleep_info["chronotype_coarse"]
chronotype.name = "chronotype"
within_ideal_bed_time = sleep_info["within_ideal_bed_time"]

In [ ]:
for index in [wakeup_source, wakeup_hour, weekend, meq, chronotype, within_ideal_bed_time]:
    if index.name not in cort_samples.reset_index().columns:
        cort_samples = cort_samples.join(index)
        cort_features = cort_features.join(index)

### Apply Codebook: Rename Index Codes, Set Index Levels, Reorder Columns

Set desired Index Order

In [ ]:
index_cols = [
    "subject",
    "condition",
    "chronotype",
    "MEQ",
    "night",
    "within_ideal_bed_time",
    "wakeup_source",
    "weekend",
    "wakeup_hour",
]

In [ ]:
cort_samples = cort_samples.reset_index().set_index(index_cols + ["sample"])
cort_samples = cort_samples[["cortisol", "time"]]

cort_features = cort_features.reset_index().set_index(index_cols + ["saliva_feature"])

display(cort_samples.head())
display(cort_features.head())

In [ ]:
codebook = dataset.codebook
cort_samples = bp.utils.dataframe_handling.apply_codebook(cort_samples, codebook)
cort_features = bp.utils.dataframe_handling.apply_codebook(cort_features, codebook)

display(cort_samples.head())
display(cort_features.head())

## Export

In [ ]:
export_path = Path("../../exports")
export_path.mkdir(exist_ok=True)

In [ ]:
cort_samples.to_csv(export_path.joinpath("cortisol_samples_cleaned.csv"))
cort_features.to_csv(export_path.joinpath("cortisol_features_cleaned.csv"))